# Merge Large Language Models with mergekit

In [8]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 1233, done.
remote: Counting objects: 100% (624/624), done.
remote: Compressing objects: 100% (214/214), done.
remote: Total 1233 (delta 524), reused 419 (delta 408), pack-reused 609
Receiving objects: 100% (1233/1233), 338.53 KiB | 10.26 MiB/s, done.
Resolving deltas: 100% (841/841), done.


In [9]:
import yaml

MODEL_NAME = "StarlingHermes-2.5-Mistral-7B-slerp"
yaml_config = """
slices:
  - sources:
      - model: berkeley-nest/Starling-LM-7B-alpha
        layer_range: [0, 32]
      - model: shahzebnaveed/NeuralHermes-2.5-Mistral-7B
        layer_range: [0, 32]
merge_method: slerp
base_model: berkeley-nest/Starling-LM-7B-alpha
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16

"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

In [3]:
from huggingface_hub import notebook_login as hfl
hfl()

In [16]:
%%capture
# Merge models
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle --write-model-card

In [17]:
!pip install -qU huggingface_hub

In [18]:
#from google.colab import userdata
from huggingface_hub import HfApi

username = "shahzebnaveed"
userdata = {'HF_TOKEN' : ''}

# Defined in the secrets tab in Google Colab
api = HfApi(token=userdata.get("HF_TOKEN"))

# api.create_repo(
#     repo_id=f"{username}/{MODEL_NAME}",
#     repo_type="model"
# )
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)

CommitInfo(commit_url='https://huggingface.co/shahzebnaveed/StarlingHermes-2.5-Mistral-7B-slerp/commit/53771500a3747e3f23a47f3bea3bc033b76c0536', commit_message='Upload folder using huggingface_hub', commit_description='', oid='53771500a3747e3f23a47f3bea3bc033b76c0536', pr_url=None, pr_revision=None, pr_num=None)